<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Sales Forecasting

### Part 4 Evaluation and Conclusion

---


## Contents:
- [Problem Statement](#Problem-Statement)
- [Gather Data](#Gather-Data)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import simplefilter
simplefilter("ignore", FutureWarning)
%matplotlib inline

In [2]:
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    ReducedForecaster,
    TransformedTargetForecaster,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting import sMAPE, smape_loss
from sktime.utils.plotting import plot_series
from sktime.forecasting.fbprophet import Prophet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
sales=pd.read_csv('../Dataset/sales_df.csv')
sales.date= pd.to_datetime(sales.date)
sales = sales.set_index('date')
sales.head()

In [ ]:
y=sales['total_sales']
z=y.copy()
y.index=y.index.to_period("D")
z_test.index.freq='D'
y_train, y_test = temporal_train_test_split(y, test_size=28)
z_train, z_test = temporal_train_test_split(z, test_size=28)
z_test.index.freq='D'
fh_p = ForecastingHorizon(z_test.index, is_relative=False)

In [3]:
result=pd.read_csv('../Dataset/result.csv')
holidays = pd.read_csv('../Dataset/holidays.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Dataset/result.csv'

In [ ]:
# sktime ensembler
forecaster = EnsembleForecaster([
        ('theta_ses', ThetaForecaster(sp=7)),
        ('theta', ThetaForecaster()),
        ("exp_ses", ExponentialSmoothing(seasonal="multiplicative", sp=7)),
        ("exp_damped", ExponentialSmoothing(trend='multiplicative', damped_trend=True, seasonal="multiplicative", sp=7)),
        #("bats", BATS(sp=7, use_trend=True, use_box_cox=False)),
        ("prophet",  Prophet(seasonality_mode='multiplicative', 
                        holidays=holidays,
                        n_changepoints=2,
                        changepoint_range=0.95,
                        changepoint_prior_scale=0.1,
                        seasonality_prior_scale=10,
                        holidays_prior_scale=10,
                        daily_seasonality=True, 
                        weekly_seasonality=True,
                        yearly_seasonality=True
                        )),
        ("lgbm", ReducedForecaster(LGBMRegressor(learning_rate = 0.1,
                            max_depth = -1, 
                            n_estimators = 50, 
                            num_leaves = 40),
                        scitype='regressor', 
                        window_length = 67
                            )),
        ("xgb", ReducedForecaster(XGBRegressor(learning_rate = 0.1,
                            max_depth = 2, 
                            n_estimators = 50),
                        scitype='regressor',
                        window_length = 80
                            )),
         ("r_forest", ReducedForecaster(RandomForestRegressor(n_estimators = 100), 
                             window_length=50, scitype='regressor',strategy="recursive"))
    ])

forecaster.fit(z_train)
y_pred = forecaster.predict(fh_p)
y_pred_es= y_pred.copy()
y_pred_es.index = y_pred.index.to_period()
plot_series(y_train[-50:], y_test, y_pred_es, labels=["y_train", "y_test", "y_pred_es"])
smape_loss(y_test, y_pred_es)

In [ ]:

fig_dims = (6, 5)
fig, ax = plt.subplots(figsize=fig_dims)
fig.suptitle('Model vs SMAPE Losss', fontsize = 16, weight = 'bold')

ax=sns.barplot(x = 'model', y = 'smape_loss', data=result);
ax.set_xlabel('Model')
ax.set_ylabel('SMAPE Loss')
ax.set_xticklabels(['Prophet','BATS','TBATS','Auto ETS','sNaive','Exponential' ,'Naive' ,'Auto Arima'], rotation='vertical', fontsize=10)
for index, row in result.iterrows():
    ax.text(row.name,row.smape_loss, round(row.smape_loss,4), color='black', ha="center")
plt.show()